<a href="https://colab.research.google.com/github/rashad101/LLM-Notebooks/blob/main/Gemma%2BLlamaindex%2BLangChain_RAG_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install required libraries**

In [ ]:
%pip install llama-index-llms-huggingface --quiet
%pip install llama-index-embeddings-huggingface --quiet
!pip install llama-index-embeddings-langchain --quiet
!pip install llama-index --quiet
!pip install -U sentence-transformers --quiet
!pip install -U transformers --quiet
!pip install -i https://pypi.org/simple/ bitsandbytes --quiet
!pip install accelerate --quiet
!pip install langchain --quiet
!pip install chromadb --quiet


**Loading Model and tokenizer**

In [2]:
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

hf_token = userdata.get('hf_token_read') # hugginface access token

# You must accept the usage terms for using google/gemma-7b in your huggigface account in the model page.
# Otherwise the model will not load

# Load the model in 8bit
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b", token=hf_token)
gemma = AutoModelForCausalLM.from_pretrained("google/gemma-7b", token=hf_token, device_map="auto", quantization_config=quantization_config, offload_folder="offload")



tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

**Download data and Create Index for retrieval**

In [3]:
!mkdir -p 'data/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham_essay.txt'
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import ServiceContext
import chromadb


# Load documents
documents = SimpleDirectoryReader("data").load_data()

# Load in a specific embedding model
embed_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

# set up ChromaVectorStore and load in data create client and a new collection
chroma_client = chromadb.EphemeralClient()
chroma_collection = chroma_client.create_collection("gemma7b-test")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# Creating an index with sentence transformer embedding model
# for better retrieval performance you may use OpenAI models

storage_context = StorageContext.from_defaults(vector_store=vector_store)


index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    embed_model=embed_model
)



--2024-02-21 22:01:35--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘data/paul_graham_essay.txt’

data/paul_graham_es 100%[===================>]  73.28K  --.-KB/s    in 0.05s   

2024-02-21 22:01:35 (1.49 MB/s) - ‘data/paul_graham_essay.txt’ saved [75042/75042]



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
question = "With what everything changed?"
retriever = index.as_retriever(similarity_top_k=3) # set retriever to retrieve top-3 results
results = retriever.retrieve(question) # performing retrieval

top_result = results[0].text # top retrieved text

prompt = f"""
Answer the Question using the provided Context. The Answer should not be no longer than 30 words.

Question: {question}

Context: {top_result}


Answer:
"""

input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = gemma.generate(**input_ids, max_new_tokens=120)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])

